# **XGBoost**
- Using data that was not annotated
- Training it on the unbalanced data
- Using avgpool
- 6 second window

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install xgboost

In [3]:
# Standard libraries
import numpy as np
import pandas as pd
import time
import os

# For audio
from IPython.display import Audio
import librosa

# For preprocessing
import xgboost as xgb
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
import tensorflow as tf

# For modeling
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, roc_auc_score, f1_score

# Operational
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
import scipy.ndimage
import pygame
import time
from scipy.signal import butter, filtfilt
import random

pygame 2.6.0 (SDL 2.28.4, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [4]:
pkl_path = '/content/drive/My Drive/Final-Year-Project/Dataset/Final-Version-of-Bird-Classification-Project/feature-extraction/Annotated/Regular/NotAveragePooled/split_features_3s_all_2D.pkl'

# Load the pickle file
with open(pkl_path, 'rb') as file:
    data = pickle.load(file)

In [5]:
train_data = data['train'].copy()
val_data = data['val'].copy()

In [6]:
train_data

{'melspectrogram': array([[[-0.01838665,  0.        , -0.00848863, ..., -0.0140964 ,
          -0.00224623, -0.00832028],
         [-0.1968215 , -0.21882324, -0.27436921, ..., -0.28576213,
          -0.22507286, -0.19316849],
         [-0.23742069, -0.24905342, -0.28791261, ..., -0.28030974,
          -0.26599282, -0.25660488],
         ...,
         [-0.99879366, -0.99876946, -0.99870896, ..., -0.99917048,
          -0.99937928, -0.99937975],
         [-0.9994275 , -0.99943322, -0.99946558, ..., -0.99974817,
          -0.99971521, -0.99969888],
         [-1.        , -1.        , -1.        , ..., -1.        ,
          -1.        , -1.        ]],
 
        [[-0.01671414,  0.        , -0.0082565 , ..., -0.01512463,
          -0.00132613, -0.0053683 ],
         [-0.17772998, -0.19626212, -0.24940659, ..., -0.30918679,
          -0.24149542, -0.20524147],
         [-0.23100212, -0.23447959, -0.25692546, ..., -0.26929191,
          -0.2456764 , -0.23258191],
         ...,
         [-0.99

In [7]:
train_labels = train_data['label'].copy()
temp = train_data.copy()
del temp['label']
tr_features = temp

In [8]:
val_labels = val_data['label'].copy()
temp = val_data.copy()
del temp['label']
v_features = temp

# Shuffle Data

In [9]:
def shuffle_data(input_label, input_features):
  input_len = len(input_label)
  np.random.seed(1826)
  input_indices = np.random.permutation(input_len)
  input_features = {key: np.array([input_features[key][i] for i in input_indices]) for key in input_features} # dictionary comprehension
  input_label = np.array([input_label[i] for i in input_indices])

  return input_label, input_features

In [10]:
train_y, train_features = shuffle_data(train_labels, tr_features)

In [11]:
val_y, val_features = shuffle_data(val_labels, v_features)

In [12]:
display(train_y.shape)
display(train_y[:15])

(5278,)

array([ 1, 16,  1,  2, 13,  3, 16, 15,  1,  0,  9, 13,  2, 17,  5],
      dtype=int32)

In [13]:
for key in train_features.keys():
  display(key)
  display(train_features[key].shape)
  display(train_features[key][0])

'melspectrogram'

(5278, 128, 259)

array([[-0.01707135,  0.        , -0.00803004, ..., -0.01309088,
        -0.00134445, -0.00628396],
       [-0.19095391, -0.20925798, -0.26611498, ..., -0.30659822,
        -0.22971404, -0.1961073 ],
       [-0.26433569, -0.28252903, -0.33811274, ..., -0.37626317,
        -0.30302411, -0.27038145],
       ...,
       [-0.98720688, -0.98624605, -0.98306841, ..., -0.98450774,
        -0.99309105, -0.99470353],
       [-0.99932593, -0.99883336, -0.99588645, ..., -0.99674553,
        -1.        , -1.        ],
       [-1.        , -1.        , -1.        , ..., -1.        ,
        -0.99780911, -0.99684298]])

'mfcc'

(5278, 20, 259)

...

(5278, 84, 259)

array([[-2.26095021e-01, -2.54480004e-01, -4.00108874e-01, ...,
        -2.03836262e-01, -3.83356214e-02, -1.49011612e-08],
       [-4.53062952e-01, -5.30015230e-01, -6.76905394e-01, ...,
        -4.46281791e-01, -2.90680707e-01, -2.05382109e-01],
       [-7.65330911e-01, -7.82241166e-01, -9.86226559e-01, ...,
        -7.43246794e-01, -5.16093910e-01, -4.64929044e-01],
       ...,
       [-1.30036249e+01, -1.90142689e+01, -1.98970470e+01, ...,
        -1.04235544e+01, -1.75874863e+01, -9.66891193e+00],
       [-1.28789825e+01, -1.69870529e+01, -1.28914795e+01, ...,
        -1.12526684e+01, -6.83810806e+00, -1.07536049e+01],
       [-1.55329523e+01, -1.89735603e+01, -1.53286657e+01, ...,
        -1.92332726e+01, -9.36328983e+00, -2.50303764e+01]])

'id'

(5278,)

'XC358628.mp3'

In [14]:
display(val_y.shape)
display(val_y[:15])

(1350,)

array([17,  2,  1,  2, 18,  2,  1,  3,  7,  2,  2, 15,  9, 15,  6],
      dtype=int32)

In [15]:
for key in val_features.keys():
  display(key)
  display(val_features[key].shape)
  display(val_features[key][0])

'melspectrogram'

(1350, 128, 259)

array([[-0.08927982, -0.07154191, -0.07405514, ..., -0.07131123,
        -0.04856302, -0.04848331],
       [-0.25275759, -0.26858278, -0.31829946, ..., -0.30957727,
        -0.2453672 , -0.21418647],
       [-0.32182292, -0.33767052, -0.38655693, ..., -0.3730203 ,
        -0.31960808, -0.29334177],
       ...,
       [-0.99583571, -0.99586676, -0.99619127, ..., -0.99568186,
        -0.99508411, -0.99514051],
       [-0.99720996, -0.99733484, -0.99788771, ..., -0.99575268,
        -0.99455928, -0.99454338],
       [-0.99826414, -0.99861173, -1.        , ..., -1.        ,
        -1.        , -1.        ]])

'mfcc'

(1350, 20, 259)

...

(1350, 84, 259)

array([[ -4.96117946,  -4.98943387,  -5.13477446, ...,  -4.10589573,
         -3.9544641 ,  -3.89248371],
       [ -5.1931474 ,  -5.2702744 ,  -5.41740439, ...,  -6.53222728,
         -5.9276817 ,  -5.43866031],
       [ -5.50660534,  -5.523381  ,  -5.72691386, ...,  -2.27492126,
         -2.23120413,  -2.28983142],
       ...,
       [-24.73274405, -60.34352413, -59.73790802, ..., -51.30912419,
        -56.29529652, -50.66915157],
       [-25.05466204, -59.58331214, -57.62388118, ..., -50.94163721,
        -52.00354656, -44.67068148],
       [-25.22672321, -62.69822785, -63.1191885 , ..., -49.59993179,
        -51.97245995, -53.07590773]])

'id'

(1350,)

'XC252756.mp3'

## **Random Forest Model**

Saving evaluation results

In [16]:
def evaluate_model(model, validation_features, val_y):
    # Predict class labels for validation set
    val_yhat_result = model.predict(validation_features)

    # Print classification report
    print('Validation classification Report \n')
    print(classification_report(val_y, val_yhat_result))

    # Get probabilities for the validation set (for AUC calculation)
    val_y_proba = model.predict_proba(validation_features)

    # Calculate AUC for multiclass classification using 'ovr' and 'weighted' average
    auc_score = roc_auc_score(val_y, val_y_proba, multi_class='ovr', average='weighted')
    print(f'AUC Score: {auc_score}')

    # Calculate F1-score with 'weighted' average for imbalanced dataset
    f1 = f1_score(val_y, val_yhat_result, average='weighted')
    print(f'F1 Score (Weighted): {f1}')

    # Store the scores in the dictionary
    val_score = {'f1': f1, 'auc': auc_score}

    # Return the scores dictionary
    return val_score

In [17]:
train_results = {}
val_results = {}

val_scores = {}

### **With all the features**

In [18]:
training_features_3D = np.concatenate((train_features['mfcc'], train_features['chroma'], train_features['cqt'], train_features['melspectrogram']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(5278, 63196)

In [19]:
validation_features_3D = np.concatenate((val_features['mfcc'], val_features['chroma'], val_features['cqt'], val_features['melspectrogram']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(1350, 63196)

Fit the model with training data

In [20]:
model = xgb.XGBClassifier(booster='gbtree', n_estimators=100, random_state=1303)
model.fit(training_features, train_y)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [21]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['all_features'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['all_features'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.7074074074074074


In [22]:
val_scores['all_features'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.79      0.74      0.76       114
           1       0.45      0.60      0.52       141
           2       0.79      0.75      0.77       271
           3       0.83      0.94      0.88        90
           4       0.50      0.06      0.10        18
           5       0.56      0.33      0.42        15
           6       0.36      0.52      0.43        23
           7       0.95      0.91      0.93        44
           8       0.78      0.76      0.77        50
           9       0.87      0.92      0.89        49
          10       0.83      0.73      0.78        48
          11       0.80      0.52      0.63        46
          12       0.82      0.91      0.86        54
          13       0.62      0.79      0.70        48
          14       0.53      0.84      0.65        38
          15       0.45      0.28      0.35       117
          16       0.88      0.96      0.92   

### **With MFCCs only**

In [23]:
training_features_3D = train_features['mfcc']
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(5278, 5180)

In [24]:
validation_features_3D = val_features['mfcc']
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(1350, 5180)

In [25]:
model = xgb.XGBClassifier(booster='gbtree', n_estimators=100, random_state=1303)
model.fit(training_features, train_y)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [26]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['mfcc'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['mfcc'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.7044444444444444


In [27]:
val_scores['mfcc'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.74      0.84      0.79       114
           1       0.43      0.62      0.51       141
           2       0.80      0.63      0.70       271
           3       0.87      0.98      0.92        90
           4       0.50      0.06      0.10        18
           5       0.00      0.00      0.00        15
           6       0.38      0.65      0.48        23
           7       0.96      1.00      0.98        44
           8       0.69      0.68      0.69        50
           9       0.96      0.90      0.93        49
          10       0.75      0.75      0.75        48
          11       0.67      0.43      0.53        46
          12       0.78      0.96      0.86        54
          13       0.56      0.75      0.64        48
          14       0.56      0.87      0.68        38
          15       0.69      0.44      0.54       117
          16       0.93      0.93      0.93   

### **With Mel-Spectrogram Only**

In [28]:
training_features_3D = train_features['melspectrogram']
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(5278, 33152)

In [29]:
validation_features_3D = val_features['melspectrogram']
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(1350, 33152)

In [30]:
model = xgb.XGBClassifier(booster='gbtree', n_estimators=100, random_state=1303)
model.fit(training_features, train_y)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [31]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['melspectrogram'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['melspectrogram'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.6607407407407407


In [32]:
val_scores['melspectrogram'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.67      0.78      0.72       114
           1       0.49      0.60      0.54       141
           2       0.76      0.71      0.74       271
           3       0.77      0.94      0.85        90
           4       0.00      0.00      0.00        18
           5       0.00      0.00      0.00        15
           6       0.27      0.39      0.32        23
           7       0.85      0.91      0.88        44
           8       0.73      0.60      0.66        50
           9       0.91      0.82      0.86        49
          10       0.69      0.52      0.60        48
          11       0.52      0.33      0.40        46
          12       0.79      0.91      0.84        54
          13       0.51      0.73      0.60        48
          14       0.50      0.79      0.61        38
          15       0.38      0.22      0.28       117
          16       0.82      1.00      0.90   

## **With Chroma Only**

In [33]:
training_features_3D = train_features['chroma']
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(5278, 3108)

In [34]:
validation_features_3D = val_features['chroma']
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(1350, 3108)

In [35]:
model = xgb.XGBClassifier(booster='gbtree', n_estimators=100, random_state=1303)
model.fit(training_features, train_y)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [36]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['chroma'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['chroma'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.5325925925925926


In [37]:
val_scores['chroma'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.51      0.53      0.52       114
           1       0.31      0.48      0.38       141
           2       0.60      0.54      0.57       271
           3       0.52      0.68      0.59        90
           4       0.00      0.00      0.00        18
           5       0.40      0.27      0.32        15
           6       0.21      0.39      0.27        23
           7       0.91      0.93      0.92        44
           8       0.33      0.26      0.29        50
           9       0.70      0.76      0.73        49
          10       0.79      0.62      0.70        48
          11       0.55      0.39      0.46        46
          12       0.76      0.81      0.79        54
          13       0.41      0.60      0.49        48
          14       0.56      0.82      0.67        38
          15       0.43      0.30      0.35       117
          16       0.70      0.47      0.57   

## **With CQT Only**

In [38]:
training_features_3D = train_features['cqt']
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(5278, 21756)

In [39]:
validation_features_3D = val_features['cqt']
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(1350, 21756)

In [40]:
model = xgb.XGBClassifier(booster='gbtree', n_estimators=100, random_state=1303)
model.fit(training_features, train_y)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [41]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['cqt'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['cqt'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.5511111111111111


In [42]:
val_scores['cqt'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.62      0.43      0.51       114
           1       0.34      0.63      0.44       141
           2       0.82      0.64      0.72       271
           3       0.55      0.64      0.59        90
           4       0.50      0.06      0.10        18
           5       0.25      0.13      0.17        15
           6       0.43      0.43      0.43        23
           7       0.98      0.95      0.97        44
           8       0.50      0.54      0.52        50
           9       0.56      0.37      0.44        49
          10       0.56      0.52      0.54        48
          11       0.53      0.41      0.46        46
          12       0.83      0.98      0.90        54
          13       0.45      0.60      0.51        48
          14       0.40      0.71      0.51        38
          15       0.12      0.09      0.11       117
          16       0.93      0.91      0.92   

## **With MFCCs and Mel-Spectrogram**

In [43]:
training_features_3D = np.concatenate((train_features['mfcc'], train_features['melspectrogram']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(5278, 38332)

In [44]:
validation_features_3D = np.concatenate((val_features['mfcc'], val_features['melspectrogram']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(1350, 38332)

In [45]:
model = xgb.XGBClassifier(booster='gbtree', n_estimators=100, random_state=1303)
model.fit(training_features, train_y)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [46]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['mfcc_melspectrogram'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['mfcc_melspectrogram'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.697037037037037


In [47]:
val_scores['mfcc_melspectrogram'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.86      0.84      0.85       114
           1       0.47      0.55      0.50       141
           2       0.76      0.73      0.74       271
           3       0.84      0.97      0.90        90
           4       0.50      0.06      0.10        18
           5       0.17      0.07      0.10        15
           6       0.33      0.43      0.38        23
           7       0.87      0.91      0.89        44
           8       0.73      0.64      0.68        50
           9       0.93      0.86      0.89        49
          10       0.76      0.67      0.71        48
          11       0.59      0.48      0.53        46
          12       0.81      0.89      0.85        54
          13       0.54      0.71      0.61        48
          14       0.47      0.79      0.59        38
          15       0.57      0.38      0.46       117
          16       0.80      0.95      0.87   

## **With MFCCs and Chroma**

In [48]:
training_features_3D = np.concatenate((train_features['mfcc'], train_features['chroma']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(5278, 8288)

In [49]:
validation_features_3D = np.concatenate((val_features['mfcc'], val_features['chroma']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(1350, 8288)

In [50]:
model = xgb.XGBClassifier(booster='gbtree', n_estimators=100, random_state=1303)
model.fit(training_features, train_y)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [51]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['mfcc_chroma'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['mfcc_chroma'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.7118518518518518


In [52]:
val_scores['mfcc_chroma'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.73      0.83      0.78       114
           1       0.42      0.50      0.46       141
           2       0.76      0.69      0.72       271
           3       0.82      0.99      0.89        90
           4       0.50      0.11      0.18        18
           5       0.17      0.07      0.10        15
           6       0.45      0.65      0.54        23
           7       0.93      0.98      0.96        44
           8       0.74      0.70      0.72        50
           9       0.92      0.90      0.91        49
          10       0.92      0.75      0.83        48
          11       0.97      0.63      0.76        46
          12       0.80      0.96      0.87        54
          13       0.51      0.81      0.62        48
          14       0.59      0.89      0.71        38
          15       0.60      0.44      0.50       117
          16       0.93      0.93      0.93   

## **With MFCCs and CQT**

In [53]:
training_features_3D = np.concatenate((train_features['mfcc'], train_features['cqt']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(5278, 26936)

In [54]:
validation_features_3D = np.concatenate((val_features['mfcc'], val_features['cqt']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(1350, 26936)

In [55]:
model = xgb.XGBClassifier(booster='gbtree', n_estimators=100, random_state=1303)
model.fit(training_features, train_y)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [56]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['mfcc_cqt'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['mfcc_cqt'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.697037037037037


In [57]:
val_scores['mfcc_cqt'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.78      0.73      0.75       114
           1       0.43      0.71      0.53       141
           2       0.85      0.65      0.73       271
           3       0.88      0.97      0.92        90
           4       1.00      0.17      0.29        18
           5       0.40      0.27      0.32        15
           6       0.39      0.61      0.47        23
           7       0.98      0.98      0.98        44
           8       0.73      0.70      0.71        50
           9       0.98      0.86      0.91        49
          10       0.84      0.77      0.80        48
          11       0.68      0.50      0.57        46
          12       0.78      0.98      0.87        54
          13       0.61      0.75      0.67        48
          14       0.46      0.84      0.59        38
          15       0.49      0.31      0.38       117
          16       0.91      0.95      0.93   

## **With Chroma and CQT**

In [58]:
training_features_3D = np.concatenate((train_features['chroma'], train_features['cqt']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(5278, 24864)

In [59]:
validation_features_3D = np.concatenate((val_features['chroma'], val_features['cqt']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(1350, 24864)

In [60]:
model = xgb.XGBClassifier(booster='gbtree', n_estimators=100, random_state=1303)
model.fit(training_features, train_y)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [61]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['chroma_cqt'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['chroma_cqt'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.6118518518518519


In [62]:
val_scores['chroma_cqt'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.69      0.47      0.56       114
           1       0.32      0.62      0.42       141
           2       0.84      0.70      0.76       271
           3       0.76      0.77      0.76        90
           4       0.50      0.06      0.10        18
           5       0.00      0.00      0.00        15
           6       0.43      0.43      0.43        23
           7       0.98      0.98      0.98        44
           8       0.42      0.52      0.46        50
           9       0.58      0.63      0.61        49
          10       0.77      0.75      0.76        48
          11       0.72      0.61      0.66        46
          12       0.74      1.00      0.85        54
          13       0.50      0.65      0.56        48
          14       0.44      0.74      0.55        38
          15       0.28      0.12      0.17       117
          16       0.83      0.89      0.86   

## **With Mel-Spectrogram and Chroma**

In [63]:
training_features_3D = np.concatenate((train_features['melspectrogram'], train_features['chroma']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(5278, 36260)

In [64]:
validation_features_3D = np.concatenate((val_features['melspectrogram'], val_features['chroma']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(1350, 36260)

In [65]:
model = xgb.XGBClassifier(booster='gbtree', n_estimators=100, random_state=1303)
model.fit(training_features, train_y)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [66]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['melspectrogram_chroma'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['melspectrogram_chroma'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.6866666666666666


In [67]:
val_scores['melspectrogram_chroma'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.66      0.77      0.71       114
           1       0.50      0.65      0.56       141
           2       0.79      0.71      0.75       271
           3       0.74      0.98      0.84        90
           4       0.33      0.06      0.10        18
           5       0.00      0.00      0.00        15
           6       0.30      0.48      0.37        23
           7       0.85      0.93      0.89        44
           8       0.74      0.56      0.64        50
           9       0.86      0.90      0.88        49
          10       0.81      0.73      0.77        48
          11       0.75      0.46      0.57        46
          12       0.81      0.81      0.81        54
          13       0.62      0.83      0.71        48
          14       0.53      0.82      0.65        38
          15       0.50      0.27      0.35       117
          16       0.76      0.93      0.84   

## **With Mel-Spectrogram and CQT**

In [68]:
training_features_3D = np.concatenate((train_features['melspectrogram'], train_features['cqt']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(5278, 54908)

In [69]:
validation_features_3D = np.concatenate((val_features['melspectrogram'], val_features['cqt']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(1350, 54908)

In [70]:
model = xgb.XGBClassifier(booster='gbtree', n_estimators=100, random_state=1303)
model.fit(training_features, train_y)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [71]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['melspectrogram_cqt'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['melspectrogram_cqt'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.6385185185185185


In [72]:
val_scores['melspectrogram_cqt'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.76      0.64      0.70       114
           1       0.37      0.55      0.45       141
           2       0.77      0.72      0.74       271
           3       0.78      0.92      0.84        90
           4       0.00      0.00      0.00        18
           5       0.00      0.00      0.00        15
           6       0.30      0.39      0.34        23
           7       0.98      0.91      0.94        44
           8       0.66      0.54      0.59        50
           9       0.86      0.76      0.80        49
          10       0.70      0.69      0.69        48
          11       0.55      0.39      0.46        46
          12       0.81      0.96      0.88        54
          13       0.57      0.67      0.62        48
          14       0.43      0.74      0.54        38
          15       0.28      0.16      0.20       117
          16       0.87      0.96      0.91   

## **With Mel-Spectrogram, MFCCs, CQT**

In [73]:
training_features_3D = np.concatenate((train_features['melspectrogram'], train_features['cqt'], train_features['mfcc']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(5278, 60088)

In [74]:
validation_features_3D = np.concatenate((val_features['melspectrogram'], val_features['cqt'], val_features['mfcc']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(1350, 60088)

In [75]:
model = xgb.XGBClassifier(booster='gbtree', n_estimators=100, random_state=1303)
model.fit(training_features, train_y)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [76]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['melspectrogram_cqt_mfcc'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['melspectrogram_cqt_mfcc'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.6881481481481482


In [77]:
val_scores['melspectrogram_cqt_mfcc'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.82      0.75      0.79       114
           1       0.40      0.58      0.48       141
           2       0.80      0.75      0.77       271
           3       0.88      0.96      0.91        90
           4       1.00      0.06      0.11        18
           5       0.62      0.33      0.43        15
           6       0.32      0.52      0.40        23
           7       0.98      0.91      0.94        44
           8       0.62      0.58      0.60        50
           9       0.92      0.90      0.91        49
          10       0.76      0.58      0.66        48
          11       0.69      0.48      0.56        46
          12       0.82      0.94      0.88        54
          13       0.55      0.75      0.63        48
          14       0.46      0.82      0.59        38
          15       0.47      0.25      0.32       117
          16       0.84      0.96      0.90   

## **With Mel-Spectrogram, MFCCs, Chroma**

In [78]:
training_features_3D = np.concatenate((train_features['melspectrogram'], train_features['chroma'], train_features['mfcc']), axis=1)
training_features = training_features_3D.reshape(training_features_3D.shape[0], -1)
training_features.shape

(5278, 41440)

In [79]:
validation_features_3D = np.concatenate((val_features['melspectrogram'], val_features['chroma'], val_features['mfcc']), axis=1)
validation_features = validation_features_3D.reshape(validation_features_3D.shape[0], -1)
validation_features.shape

(1350, 41440)

In [80]:
model = xgb.XGBClassifier(booster='gbtree', n_estimators=100, random_state=1303)
model.fit(training_features, train_y)

XGBClassifier(base_score=None, booster='gbtree', callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=None, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=None, max_leaves=None,
              min_child_weight=None, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, objective='multi:softprob', ...)

In [81]:
print('Training accuracy:', model.score(training_features, train_y))
train_results['melspectrogram_chroma_mfcc'] = model.score(training_features, train_y)

print('Validation accuracy:', model.score(validation_features, val_y))
val_results['melspectrogram_chroma_mfcc'] = model.score(validation_features, val_y)

Training accuracy: 1.0
Validation accuracy: 0.7007407407407408


In [82]:
val_scores['melspectrogram_cqt_mfcc'] = evaluate_model(model=model, validation_features=validation_features, val_y=val_y)

Validation classification Report 

              precision    recall  f1-score   support

           0       0.78      0.82      0.80       114
           1       0.51      0.60      0.55       141
           2       0.78      0.73      0.75       271
           3       0.84      0.98      0.90        90
           4       0.00      0.00      0.00        18
           5       0.29      0.13      0.18        15
           6       0.35      0.48      0.41        23
           7       0.82      0.93      0.87        44
           8       0.69      0.62      0.65        50
           9       0.90      0.88      0.89        49
          10       0.81      0.71      0.76        48
          11       0.74      0.54      0.62        46
          12       0.83      0.89      0.86        54
          13       0.54      0.73      0.62        48
          14       0.47      0.82      0.60        38
          15       0.47      0.31      0.37       117
          16       0.84      0.93      0.88   

# Review results from all models

In [83]:
train_results_df = pd.DataFrame(list(train_results.items()), columns=['Features', 'Train_Accuracy']).round(3)
val_results_df = pd.DataFrame(list(val_results.items()), columns=['Features', 'Val_Accuracy']).round(3)

result_df = train_results_df.merge(val_results_df, on='Features')
result_df = result_df.sort_values('Features')
result_df

,Features,Train_Accuracy,Val_Accuracy
0,all_features,1.0,0.707
3,chroma,1.0,0.533
8,chroma_cqt,1.0,0.612
4,cqt,1.0,0.551
2,melspectrogram,1.0,0.661
9,melspectrogram_chroma,1.0,0.687
12,melspectrogram_chroma_mfcc,1.0,0.701
10,melspectrogram_cqt,1.0,0.639
11,melspectrogram_cqt_mfcc,1.0,0.688
1,mfcc,1.0,0.704


In [84]:
val_scores_df = pd.DataFrame([(key, value['f1'], value['auc']) for key, value in val_scores.items()],
                             columns=['Features', 'F1_Score', 'AUC_Score']).round(3)

val_scores_df = val_scores_df.sort_values('Features')
print(val_scores_df)

                   Features  F1_Score  AUC_Score
0              all_features     0.702      0.955
3                    chroma     0.530      0.901
8                chroma_cqt     0.606      0.940
4                       cqt     0.550      0.918
2            melspectrogram     0.648      0.943
9     melspectrogram_chroma     0.676      0.950
10       melspectrogram_cqt     0.632      0.938
11  melspectrogram_cqt_mfcc     0.693      0.957
1                      mfcc     0.700      0.955
6               mfcc_chroma     0.708      0.960
7                  mfcc_cqt     0.697      0.957
5       mfcc_melspectrogram     0.691      0.950
